In [2]:
library(data.table)
library(dtplyr)
library(dplyr, warn.conflicts = FALSE)
library(lubridate)
library(tidyr)

In [3]:
meta <- fread(snakemake@input$data)
nrow(meta)

In [4]:
ids <- fread(snakemake@input$ids, header = F)
ids <- unlist(ids$V1)
meta <- meta %>%
    filter(taxon %in% ids) %>%
    as.data.frame()
nrow(meta)

In [5]:
lineage4focus=snakemake@params$id

In [7]:
df_focus = meta %>%
    lazy_dt() %>%
    mutate(date = date(date)) %>%
    mutate(week = lubridate::floor_date(date, "week")) %>%
    as.data.table() 
head(df_focus)

In [11]:

df_focus_per_week <- df_focus %>%
    lazy_dt() %>%
    group_by(week) %>%
    summarise(N=n(), IDs=list(taxon)) %>%
    as.data.table()

In [16]:
ct = 0
dir.create(snakemake@output$cluster, showWarnings = F)
for (lini in df_focus_per_week$IDs) {
    ct = ct + 1
    df = data.table(id = lini)
    fname = paste(snakemake@output[[1]],"/week",ct,".txt",collapse="", sep="")
    fwrite(file = fname, x = df, col.names = F, row.names = F)
}

